In [1]:
import numpy as np 
import pandas as pd
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
import time
import pickle
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

seed = 42
CONTEXT_WINDOW = (-5, 5)

In [2]:
data = pd.read_csv('train_data.csv', encoding = 'utf-8-sig', sep = ',',index_col=0)
data = data.fillna('')
data.head()

,Text_-5,Lemma_-5,POS_-5,Target_-5,Text_-4,Lemma_-4,POS_-4,Target_-4,Text_-3,Lemma_-3,...,POS_2,Text_3,Lemma_3,POS_3,Text_4,Lemma_4,POS_4,Text_5,Lemma_5,POS_5
0,,,,,,,,,,,...,NOUN,воды,вода,NOUN,диалог,диалог,NOUN,москва,москва,NOUN
1,,,,,,,,,,,...,NOUN,диалог,диалог,NOUN,москва,москва,NOUN,",",",",PNCT
2,,,,,,,,,,,...,NOUN,москва,москва,NOUN,",",",",PNCT,изд-во,изд-в,PREP
3,,,,,,,,,л.петрушевская,л.петрушевский,...,NOUN,",",",",PNCT,изд-во,изд-в,PREP,``,``,UNKN
4,,,,,л.петрушевская,л.петрушевский,NOUN,O,.,.,...,PNCT,изд-во,изд-в,PREP,``,``,UNKN,искусство,искусство,NOUN


In [3]:
target = pd.read_csv('train_target.csv',encoding = 'utf-8-sig', sep = ';' )
y = np.array(target['Target']).reshape(-1,)

In [4]:
# в качестве теста - взять несколько текстов
test_texts = {'Puzhaeva_Гоголь. Игроки.txt',
             'Petrushevskaya_uroki_muzyki_ev_prov.txt',
             'Yad_EV_prov.txt'}
criterion = data['Text_id'].map(lambda x: x in test_texts)
test_indices = data[criterion].index.tolist()
X_test = data.loc[test_indices,:]
y_test = np.array(target.loc[test_indices,:]).reshape(-1,)
X_train = data.drop(test_indices)
y_train = np.array(target.drop(test_indices)).reshape(-1,)
X_train.shape, X_test.shape

((4982, 40), (0, 40))

In [5]:
type(data.iloc[0]['Text_-5'])

str

In [6]:
def vectorize(data,X_train,X_test,colname):
    print(colname)
    label_enc = LabelEncoder()
    onehot_enc = OneHotEncoder()
    if colname == 'Target':
        label_data = label_enc.fit_transform(['B','I','L','O','U',''])
        cols_to_transform = [colname+'_'+str(i) for i in range(CONTEXT_WINDOW[0],0)]
    else:
        label_data = label_enc.fit_transform(list(data[colname].unique())+[''])
        cols_to_transform = [colname] + [colname+'_'+str(i) for i in range(CONTEXT_WINDOW[0],CONTEXT_WINDOW[1]+1) if i]
    onehot_enc.fit(label_data.reshape(-1, 1))
    #count_train = hstack([onehot_enc.transform(label_enc.transform(X_train[col]).reshape(-1, 1)) for col in cols_to_transform])
    #count_test = hstack([onehot_enc.transform(label_enc.transform(X_test[col]).reshape(-1, 1)) for col in cols_to_transform])
    count_texts = hstack([onehot_enc.transform(label_enc.transform(data[col]).reshape(-1, 1)) for col in cols_to_transform])
    return [count_texts] #,count_train,count_test

In [11]:
cols_to_vect = ['Text','Lemma','POS','Target']
sparse_data = [vectorize(data,X_train,X_test,x) for x in cols_to_vect]
data_vect = hstack([x[0] for x in sparse_data]+[np.array(data['Word_num'])[:,None]])
#train_vect = hstack([x[1] for x in sparse_data]+csr_matrix(X_train['Word_num']))
#test_vect = hstack([x[2] for x in sparse_data]+csr_matrix(X_test['Word_num']))
data_vect.shape #,train_vect.shape,test_vect.shape

Text
Lemma
POS
Target


(4982, 29433)

In [9]:
# кроссвалидация по пяти фолдам
skf = StratifiedKFold(n_splits = 5, random_state = seed)

In [12]:
logit = LogisticRegression(random_state=seed)
cv_logit = cross_val_predict(logit, data_vect, y, cv = skf, verbose = 5)
print('Accuracy:')
print(accuracy_score(y, cv_logit))
print(classification_report(y, cv_logit))

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


Accuracy:
0.9995985547972702
             precision    recall  f1-score   support

          B       1.00      0.86      0.92         7
          I       1.00      1.00      1.00         9
          L       1.00      0.86      0.92         7
          O       1.00      1.00      1.00      4959

avg / total       1.00      1.00      1.00      4982



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.7s finished
